# Neural Net From Scratch

## Imports

In [5]:
import numpy as np
from random import random, seed

seed(0)

## Model

### Activation functions

In [1]:
class Linear():
    @staticmethod
    def f(x):
        return x
    @staticmethod
    def df(x):
        return x

class Relu():
    @staticmethod
    def f(x):
        return x * (x > 0)
    @staticmethod
    def df(x):
        return 1 * (x > 0)

class Sigmoid():
    @staticmethod
    def f(x):
        return 1.0 / (1.0 + np.exp(-x))
    @staticmethod
    def df(x):
        return Sigmoid.f(x) * (1 - Sigmoid.f(x))

### Loss functions

In [2]:
class MSE():
  @staticmethod
  def f(target, output):
    return np.mean((target - output) **2)
  
  @staticmethod 
  def df(target, output):
    return 2 * (target - output) / np.size(output)

class Crossentropy():
  @staticmethod
  def f(target, output):
    return np.mean(-target * np.log(output) - (1 - target) * np.log(1 - output))
  
  @staticmethod 
  def df(target, output):
    return ((1 -  target) / (1 - output) - target/output) / np.size(target)  

### Dense layer

In [3]:
class Layer:
    def __init__(self, neurons, activation=Linear):
        self.neurons = neurons
        self.activation = activation
        self.weights = None
        self.biases = None
        self.lr = 0.02 # Snelheid waarmee de model traint -> staat nu hardcoded

    def initialize(self, prev_neurons):
        self.weights = np.random.normal(0.0, 1.0, (self.neurons, prev_neurons))
        self.biases = np.random.normal(size=(self.neurons, 1))

    def feed_forward(self, input_array):
        # Input en output opslaan -> zijn nodig voor gradient descent
        self.inputs = input_array 

        # Y = W * I + B (matrix * vector)
        self.output = self.activation.f(np.dot(self.weights, self.inputs) + self.biases) 
        return self.output

    def gradient_descent(self, errors):
        delta_weights = self.lr * np.dot((errors * self.activation.df(self.output)), self.inputs.T)
        delta_biases = self.lr * errors * self.activation.df(self.output)

        self.weights = np.add(self.weights, delta_weights)
        self.biases = np.add(self.biases, delta_biases)

### Neurale netwerk

In [102]:
class Sequential():
    def __init__(self):
        self.layers = []

    def add(self, layer):
        # Initialiseer de laag met weights en biases alleen als het niet de input laag is.
        if len(self.layers) > 0:
            prev_neurons = self.layers[-1].neurons
            layer.initialize(prev_neurons)
        
        self.layers.append(layer)
        

    def predict(self, input_array, transposed = False):
        assert isinstance(input_array, np.ndarray)
        
        output = input_array
        
        if not transposed:
            output = output.T

        for lay in self.layers[1:]:
            output = lay.feed_forward(output)
        return output

    def fit(self, input_array: np.ndarray ,output_array: np.ndarray, loss_function=MSE, epochs=10):
        assert isinstance(input_array, np.ndarray)
        assert isinstance(output_array, np.ndarray)


        for epoch in range(epochs):
            sum_errors = 0

            for x, y in zip(input_array, output_array):
                
                # feedforward
                prediction = self.predict(x.reshape(-1,1), transposed = True) # Reshape maakt ndmin 2 en transposed het al

                sum_errors += loss_function.f(y, prediction) # houdt mse bij om het te visualiseren

                # backprop
                error = np.array(loss_function.df(y, prediction), ndmin=2) # gebruik afgeleide om weten of we de gewichten moeten verhogen of verlagen

                for lay in reversed(self.layers[1:]):
                    lay.gradient_descent(error)
                    error = np.dot(lay.weights.T, error)           
            sum_errors /= len(input_array)

            print(f"Epoch {epoch+1}/{epochs}:")
            print(f"Error: {sum_errors}\n")
        print(f"\nFinished Training\n{'=' * 50}")



## Testing

#### Regressie

Getallen optellen

e.g. array van [0.23, 0.56] = 0.79

In [106]:
X = np.array([[random() for _ in range(2)] for _ in range(1000)])
y = np.array([i[0] + i[1] for i in X])

In [107]:
model = Sequential()

model.add(Layer(2, activation=Relu))
model.add(Layer(5, activation=Relu))
model.add(Layer(1, activation=Relu))

In [108]:
model.fit(X, y, epochs=10, loss_function=MSE)

Epoch 1/10:
Error: 0.026860559535293045

Epoch 2/10:
Error: 0.0010098502447980296

Epoch 3/10:
Error: 0.0005315250145749669

Epoch 4/10:
Error: 0.00034881161059591083

Epoch 5/10:
Error: 0.0002233128114892738

Epoch 6/10:
Error: 0.00014225052680706985

Epoch 7/10:
Error: 9.485987659344488e-05

Epoch 8/10:
Error: 6.572407803129824e-05

Epoch 9/10:
Error: 4.758961195816925e-05

Epoch 10/10:
Error: 3.5530272897235897e-05


Finished Training


In [109]:
model.predict(np.array([[0.53, 0.11]]))

array([[0.64134728]])

In [110]:
test_set = np.array([[3, 8], [50, 32],[0.4, 0.4], [0.27, 0.32], [0.1, 0.13], [0.76, 0.22], [0.2, 0.3]]) # 11, 82, 0.8, 0.59, 0.23, 0.98, 0.50
model.predict(test_set)

array([[10.96512561, 81.61924627,  0.80133382,  0.59232758,  0.22858303,
         0.97967495,  0.50279986]])

### Binary classification

#### AND

In [111]:
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([0,0,0,1])

In [112]:
model = Sequential()

model.add(Layer(2, activation=Relu))
model.add(Layer(5, activation=Relu))
model.add(Layer(3, activation=Relu))
model.add(Layer(1, activation=Sigmoid))


model.fit(X, y, epochs=1000)

Epoch 1/1000:
Error: 0.21512392017244542

Epoch 2/1000:
Error: 0.2033294953097847

Epoch 3/1000:
Error: 0.19184176330825545

Epoch 4/1000:
Error: 0.17282561228683962

Epoch 5/1000:
Error: 0.14843159089444602

Epoch 6/1000:
Error: 0.13058902300446998

Epoch 7/1000:
Error: 0.12845533492007977

Epoch 8/1000:
Error: 0.12644370063150268

Epoch 9/1000:
Error: 0.1245425492694418

Epoch 10/1000:
Error: 0.12274018800427297

Epoch 11/1000:
Error: 0.12102523542312671

Epoch 12/1000:
Error: 0.11938685044379163

Epoch 13/1000:
Error: 0.11781489885667672

Epoch 14/1000:
Error: 0.11630005786858427

Epoch 15/1000:
Error: 0.11483386533903439

Epoch 16/1000:
Error: 0.11340872390283939

Epoch 17/1000:
Error: 0.11201787134841504

Epoch 18/1000:
Error: 0.1106553281419598

Epoch 19/1000:
Error: 0.10931583151807242

Epoch 20/1000:
Error: 0.1079947636455387

Epoch 21/1000:
Error: 0.10668807941356986

Epoch 22/1000:
Error: 0.10539223760816693

Epoch 23/1000:
Error: 0.10410413777813107

Epoch 24/1000:
Error: 0.

In [113]:
model.predict(np.array([[1,1], [1, 0], [0, 0], [0, 1]]))

array([[9.96878596e-01, 1.75926806e-03, 6.75412575e-06, 2.88736203e-03]])

#### OR

In [91]:
y = np.array([0,1,1,1])
model.fit(X, y, epochs=1000)

Epoch 1/1000:
Error: 0.00011438706313885999

Epoch 2/1000:
Error: 9.441413162993735e-05

Epoch 3/1000:
Error: 7.925632870803307e-05

Epoch 4/1000:
Error: 6.748564216027268e-05

Epoch 5/1000:
Error: 5.816636114114441e-05

Epoch 6/1000:
Error: 5.0664604291578136e-05

Epoch 7/1000:
Error: 4.4538620095808915e-05

Epoch 8/1000:
Error: 3.947294233687574e-05

Epoch 9/1000:
Error: 3.523744442774333e-05

Epoch 10/1000:
Error: 3.166108349037033e-05

Epoch 11/1000:
Error: 3.05503048893419e-05

Epoch 12/1000:
Error: 2.9775843517965507e-05

Epoch 13/1000:
Error: 2.9029652734661135e-05

Epoch 14/1000:
Error: 2.831043546400832e-05

Epoch 15/1000:
Error: 2.761696587596344e-05

Epoch 16/1000:
Error: 2.6948084838577688e-05

Epoch 17/1000:
Error: 2.630269570033483e-05

Epoch 18/1000:
Error: 2.567976037545462e-05

Epoch 19/1000:
Error: 2.5078295707868934e-05

Epoch 20/1000:
Error: 2.4497370091721472e-05

Epoch 21/1000:
Error: 2.393610032817882e-05

Epoch 22/1000:
Error: 2.339364870010898e-05

Epoch 23/100

In [92]:
model.predict(np.array([[1,1], [1, 0], [0, 0], [0, 1]]))

array([[9.99999612e-01, 9.99849092e-01, 7.04610881e-04, 9.99329606e-01]])

#### XOR

Dit keer met wat minder epochs en minder layers om verschilt te zien

In [119]:
y = np.array([0, 1,1,0])

In [120]:
model = Sequential()

model.add(Layer(2, activation=Relu))
model.add(Layer(3, activation=Relu))
model.add(Layer(1, activation=Sigmoid))

model.fit(X, y, epochs=150)

Epoch 1/150:
Error: 0.2594735963111816

Epoch 2/150:
Error: 0.2585824516542259

Epoch 3/150:
Error: 0.2577952240478827

Epoch 4/150:
Error: 0.257094833031118

Epoch 5/150:
Error: 0.2564669152621959

Epoch 6/150:
Error: 0.25589939815347335

Epoch 7/150:
Error: 0.2553821361146195

Epoch 8/150:
Error: 0.2549066019613441

Epoch 9/150:
Error: 0.2544656262261179

Epoch 10/150:
Error: 0.25405317762068136

Epoch 11/150:
Error: 0.25367749644404003

Epoch 12/150:
Error: 0.2533221760274474

Epoch 13/150:
Error: 0.25298385823821357

Epoch 14/150:
Error: 0.25265952602014113

Epoch 15/150:
Error: 0.2523466349063419

Epoch 16/150:
Error: 0.2520430365825017

Epoch 17/150:
Error: 0.25174691490859263

Epoch 18/150:
Error: 0.25145673235772215

Epoch 19/150:
Error: 0.25117118516244114

Epoch 20/150:
Error: 0.25088916573889253

Epoch 21/150:
Error: 0.25060973119465496

Epoch 22/150:
Error: 0.25033207692344217

Epoch 23/150:
Error: 0.25005551445477286

Epoch 24/150:
Error: 0.24977945286440484

Epoch 25/150:

In [121]:
model.predict(np.array([[1,1], [1, 0], [0, 0], [0, 1]]))

array([[0.46181592, 0.65144467, 0.35942641, 0.50502534]])

Je kan dus predicten met hoeveel dingen je maar ook wilt

In [123]:
model.predict(np.array([[1, 0]]))

array([[0.65144467]])